# IIND 4115 - Midterm Project
- Paula Artunduaga.
- Iván García Laverde.

In [2]:
!pip install ortools

In [1]:

#Copyright 2020 Ivan Felipe García Laverde
#
#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at
#
#    http://www.apache.org/licenses/LICENSE-2.0
#
#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.


In [9]:
%%time
import pandas as pd
import numpy as np
import pygeohash as gh
import folium

from haversine import haversine, Unit

data = pd.read_csv('rappi_data.csv', sep=";")
data.head()

Wall time: 3.12 s


,instance_id,order_id,courier_id,store_lat,store_lng,user_lat,user_lng,courier_lat,courier_lng,vehicle_type,store_eta_lb,store_eta_ub,user_eta_lb,user_eta_ub,store_service_time,user_service_time,geohash
0,b42ca81f3,102024749,27990,4.65357,-74.06046,4.670465,-74.085594,4.661303,-74.062622,motorcycle,0,600,0,600,360,480,d2g66xnv
1,b42ca81f3,102024749,60988,4.65357,-74.06046,4.670465,-74.085594,4.647719,-74.056915,car,0,600,0,600,360,480,d2g66xnv
2,b42ca81f3,102024749,83294,4.65357,-74.06046,4.670465,-74.085594,4.636381,-74.062054,motorcycle,0,600,0,600,360,480,d2g66xnv
3,b42ca81f3,102024749,103080,4.65357,-74.06046,4.670465,-74.085594,4.631058,-74.057548,motorcycle,0,600,0,600,360,480,d2g66xnv
4,b42ca81f3,102024749,162221,4.65357,-74.06046,4.670465,-74.085594,4.646868,-74.061598,motorcycle,0,600,0,600,360,480,d2g66xnv


## 1. Exploratory Analysis

In [3]:
print("N° data (rows): ", len(data.index))
print("N° columns: ", len(data.columns))
print("Columns data types:\n")
print(data.dtypes)

N° data (rows):  1048575
N° columns:  17
Columns data types:

instance_id            object
order_id                int64
courier_id              int64
store_lat             float64
store_lng             float64
user_lat              float64
user_lng              float64
courier_lat           float64
courier_lng           float64
vehicle_type           object
store_eta_lb            int64
store_eta_ub            int64
user_eta_lb             int64
user_eta_ub             int64
store_service_time      int64
user_service_time       int64
geohash                object
dtype: object


### 1.1 Number of rows in the file for each instance

In [33]:
print("N° of instances: ", len(data["instance_id"].unique()))
print("N° data/instance:")
print(data["instance_id"].value_counts())

N° of instances:  60
N° data/instance:
19df4db80    125029
dc3ec3fb7    117189
fad5786f9    100850
f5368b7b5     87885
8350eb5fe     74264
c0b211663     37801
4ee341f44     37124
cca353708     33189
a1a38409b     32047
050314e7f     30840
92915c3c7     28851
e6c93ac6e     23133
194ce5835     22347
ce48f383f     20499
4d0f856de     20329
b6c910c69     18476
ef66176ba     17777
8478904aa     16885
7d49e2e20     16179
e75c638a8     13902
f5089aca0     13658
b42ca81f3     13198
624d376c6     12983
a639a6b83     12922
c3ae82077     12519
05b0d2176     11105
1aeb9352b     10457
7a3f74d3b      9764
e7b29056e      7766
f680ed4ff      7751
6d41d2d0f      5713
9fe226dcc      5120
7bcb8cc32      4798
a366cc689      4533
a3fb6cd9b      4117
3fc9f4c81      3970
642246e6f      3746
99a10db5d      3320
7636e8513      3300
b3ebe98a3      3260
bd3821c33      3108
21c1b3e52      2539
31a5323b9      2529
039812c88      2022
57e549baa      1845
674d0e30e      1241
0a7ac04ce       922
e973a0ed0       828
d

### 1.2 First test with the smallest instance

In [10]:
"""
We select a test instance: the one with the minimum amount of data
"""
print("Available testing instances: ")
print("1. 3a1e169b8 (# rows: 164)")
print("2. 0a7ac04ce (# rows: 922)")
print("3. 21c1b3e52 (# rows: 2539)")
print("4. 6d41d2d0f (# rows: 5713)")
print("5. 8478904aa (# rows: 16885)")
print("\n... Please don't try to test higher # rows instances, the model is not fully optimized yet.")

inst_num = int(input("Select the instance you want to test: "))
def set_up_instance(instance_num):
    if instance_num == 1:
        instance = "3a1e169b8"
        test_instance = data[data["instance_id"] == instance]
        return test_instance.copy()
    if instance_num == 2:
        instance = "0a7ac04ce"
        test_instance = data[data["instance_id"] == instance]
        return test_instance.copy()
    if instance_num == 3:
        instance = "21c1b3e52"
        test_instance = data[data["instance_id"] == instance]
        return test_instance.copy()
    if instance_num == 4:
        instance = "6d41d2d0f"
        test_instance = data[data["instance_id"] == instance]
        return test_instance.copy()
    if instance_num == 5:
        instance = "8478904aa"
        test_instance = data[data["instance_id"] == instance]
        return test_instance.copy()
    
test_instance = set_up_instance(inst_num)
test_instance.head()

Available testing instances: 
1. 3a1e169b8 (# rows: 164)
2. 0a7ac04ce (# rows: 922)
3. 21c1b3e52 (# rows: 2539)
4. 6d41d2d0f (# rows: 5713)
5. 8478904aa (# rows: 16885)

... Please don't try to test higher # rows instances, the model is not fully optimized yet.


Select the instance you want to test:  1


,instance_id,order_id,courier_id,store_lat,store_lng,user_lat,user_lng,courier_lat,courier_lng,vehicle_type,store_eta_lb,store_eta_ub,user_eta_lb,user_eta_ub,store_service_time,user_service_time,geohash
622445,3a1e169b8,66113201,232622,-8.086116,-34.887424,-8.113902,-34.898851,-8.064354,-34.895429,motorcycle,0,600,0,600,480,240,7nx1ypgy
622446,3a1e169b8,66113201,680973,-8.086116,-34.887424,-8.113902,-34.898851,-8.116031,-34.896701,car,0,600,0,600,480,240,7nx1ypgy
622447,3a1e169b8,66113201,722928,-8.086116,-34.887424,-8.113902,-34.898851,-8.116173,-34.896588,motorcycle,0,600,0,600,480,240,7nx1ypgy
622448,3a1e169b8,66113201,799965,-8.086116,-34.887424,-8.113902,-34.898851,-8.058394,-34.898991,motorcycle,0,600,0,600,480,240,7nx1ypgy
622449,3a1e169b8,66113201,840347,-8.086116,-34.887424,-8.113902,-34.898851,-8.074031,-34.913245,motorcycle,0,600,0,600,480,240,7nx1ypgy


### 1.3 Number of orders, stores and couriers (for the test instance)

In [11]:
"""
How is the relation between users/orders <-> stores  <-> couriers?
"""
print("Number of users/orders: ", len(test_instance["order_id"].value_counts()))
print("Number of stores lats: ", len(test_instance["store_lat"].value_counts()))
print("Number of stores longs: ", len(test_instance["store_lng"].value_counts()))
print("Number of couriers: ", len(test_instance["courier_id"].value_counts()))

Number of users/orders:  22
Number of stores lats:  14
Number of stores longs:  14
Number of couriers:  53


#### <em>Observation: Different users can have an order with the same store</em>

### 1.4 Stores locations

In [12]:
"""
Zip a list of tuples for the stores for the unique lats and lngs
"""
stores = test_instance[["store_lat", "store_lng"]].groupby(["store_lat", "store_lng"]).count().reset_index()
store_dirs = list(zip(stores['store_lat'].to_list(), stores['store_lng'].to_list()))
store_dirs_dict = dict(zip(stores.index, store_dirs))
"""
Structure of the stores dictionary
    {
        -key | store_id : val | (store_lat, store_lng)
        .
        .
        .
    }
""" 
maps = folium.Map(location=(store_dirs_dict[1]), zoom_start=11.5)
for key in store_dirs_dict.keys():
    folium.Marker(
        location=store_dirs_dict[key],
        tooltip=(f"Store {key}"),
        popup=folium.Popup(max_width=140),
        icon=folium.Icon(color="gray")
        ).add_to(maps)

### 1.5 Orders locations

In [13]:
orders = test_instance.groupby(["order_id","user_lat","user_lng"]).count().reset_index()
orders = orders[["order_id","user_lat","user_lng"]]
orders_dirs = list(zip(orders["user_lat"].to_list(), orders["user_lng"].to_list()))
orders_dirs_dict = dict(zip(orders["order_id"].to_list(),orders_dirs))
"""
Structure of the stores dictionary
    {
        -key | order_id : val | (order_lat, order_lng)
        .
        .
        .
    }
"""
for key in orders_dirs_dict.keys():
    folium.Marker(
        location=orders_dirs_dict[key],
        tooltip=(f"Order {key}"),
        popup=folium.Popup(max_width=140),
        icon=folium.Icon(color="blue")
        ).add_to(maps)


### 1.6 Couriers locations

In [14]:
couriers = test_instance.groupby(["courier_id","courier_lat","courier_lng","vehicle_type"]).count()

# Couriers df is the one who is going to help me with the transportation classification of the cost matrix
couriers_df = pd.DataFrame.from_records(couriers.index, columns=['courier_id', 'courier_lat', 'courier_lng', 'vehicle_type'])
couriers_dirs = list(zip(couriers_df["courier_lat"].to_list(), couriers_df["courier_lng"].to_list()))
couriers_dirs_dict = dict(zip(couriers_df["courier_id"].to_list(), couriers_dirs))


for key in couriers_dirs_dict.keys():
    folium.Marker(
        location=couriers_dirs_dict[key],
        tooltip=(f"Courier {key}"),
        popup=folium.Popup(max_width=140),
        icon=folium.Icon(color="lightred")
        ).add_to(maps)

### 1.7 Map with the instance visualization
 - Gray: stores
 - Blue: orders
 - Light red: couriers

In [15]:
maps ##TODO, put legend

### 1.8 Set up of distances (courier -> store) and (store -> user) and creation of courier geohash for future utility:

In [8]:
%%time 
all_posible = test_instance.copy()

#Creation of stores id in the instance dataframe
_stores = all_posible[["store_lat", "store_lng"]].groupby(["store_lat", "store_lng"]).count().reset_index()
_stores['store_id'] = _stores.index

#After running this loop, each row in the test_instance dataframe has its 'store_id' updated correctly
for value in _stores.itertuples():
    """
    value[0] : Row index
    value[1] : store lat
    value[2] : store lng
    value[3] : store id
    """
    #Use Pandas vectorization to update the rows with its according ID
    #Check this link: 
    # https://towardsdatascience.com/how-to-make-your-pandas-loop-71-803-times-faster-805030df4f06
    all_posible.loc[((all_posible['store_lat'] == value[1]) & (all_posible['store_lng'] == value[2])), "store_id"] = value[3]+1

# Lets find out which stores attend to the same order/user and make them different 
# ---->>>> to make FEASIBLE the problem
_cols = ['order_id', 'store_id']
orders_same_store = all_posible[_cols].groupby(_cols).count().reset_index()
orders_same_store#['store_id'].value_counts()
for i, row in orders_same_store.iterrows():
    all_posible.loc[((all_posible['order_id'] == row['order_id']) & (all_posible['store_id'] == row['store_id'])), "store_id"] = i+1
    
#Creation of couriers
def courier_geohash(courier_lat, courier_lng):
    return gh.encode(courier_lat, courier_lng, precision=5) #closest digit precision for future utility
  
#Compute haversine distance
all_posible['havrs_courier_store'] = all_posible.apply(lambda row : haversine((row['store_lat'], row['store_lng']), (row['courier_lat'], row['courier_lng']), unit='km'), axis=1)
all_posible['havrs_store_user'] = all_posible.apply(lambda row : haversine((row['store_lat'], row['store_lng']), (row['user_lat'], row['user_lng']), unit='km'), axis=1)
all_posible['courier_geohash'] = all_posible.apply(lambda row: courier_geohash(row['courier_lat'], row['courier_lng']), axis = 1)

all_posible.head()

Wall time: 274 ms


,instance_id,order_id,courier_id,store_lat,store_lng,user_lat,user_lng,courier_lat,courier_lng,vehicle_type,...,store_eta_ub,user_eta_lb,user_eta_ub,store_service_time,user_service_time,geohash,store_id,havrs_courier_store,havrs_store_user,courier_geohash
622445,3a1e169b8,66113201,232622,-8.086116,-34.887424,-8.113902,-34.898851,-8.064354,-34.895429,motorcycle,...,600,0,600,480,240,7nx1ypgy,1.0,2.575396,3.335894,7nx4j
622446,3a1e169b8,66113201,680973,-8.086116,-34.887424,-8.113902,-34.898851,-8.116031,-34.896701,car,...,600,0,600,480,240,7nx1ypgy,1.0,3.479635,3.335894,7nx1v
622447,3a1e169b8,66113201,722928,-8.086116,-34.887424,-8.113902,-34.898851,-8.116173,-34.896588,motorcycle,...,600,0,600,480,240,7nx1ypgy,1.0,3.491123,3.335894,7nx1v
622448,3a1e169b8,66113201,799965,-8.086116,-34.887424,-8.113902,-34.898851,-8.058394,-34.898991,motorcycle,...,600,0,600,480,240,7nx1ypgy,1.0,3.335226,3.335894,7nx4j
622449,3a1e169b8,66113201,840347,-8.086116,-34.887424,-8.113902,-34.898851,-8.074031,-34.913245,motorcycle,...,600,0,600,480,240,7nx1ypgy,1.0,3.144220,3.335894,7nx4j


## 2.0 Construction of the problem parameters
### 2.1 Construction of time costs matrix

<em>Note: in this first step we restrict the problem to only the motorcycle feasible routes.
The main idea for the algorithm is to iterate over the best possible options in this logic:
1. Solve the problem for the motorcycle's couriers -> check what orders couldn't be attended.
2. Try to solve the not attended orders with the cars -> check what orders couldn't be attended.
3. Try to solve the not attended orders with the bicycles -> check what orders couldnt be attended
4. Report the overall solution with the overall not attended orders penalization</em>

In [17]:
%%time
# Nodes ids
all_posible = all_posible.loc[(all_posible['vehicle_type'] == 'motorcycle')].copy()
sts_ids = list(all_posible["store_id"].value_counts().index) #Stores ids
ords_ids = list(all_posible["order_id"].value_counts().index) #Orders ids
cours_ids = list(all_posible["courier_id"].value_counts().index) #Couriers ids

#Initialization of Costs Matrix
cost_matrix_ids = cours_ids + sts_ids + ords_ids
cost_matrix_moto = pd.DataFrame(2401, index=cost_matrix_ids, columns=cost_matrix_ids) #Creates a DataFrame full of 2401 (max number)

#Submatriz of couriers:
for cour_id in cours_ids:
    cost_matrix_moto.loc[cour_id,cour_id] = 0
    
#Submatrix of couriers (i) vs stores (j) 
#(note that the transposed matrix of  stores vs couriers must be full of 2401, since there is no way back from stores to couriers)
_cols = ['courier_id','store_id','havrs_courier_store']
couriers_stores = all_posible[_cols].groupby(_cols).count().reset_index()
for i, row in couriers_stores.iterrows():
    #Moto distances
    moto_havrs_courier_store = round((row['havrs_courier_store']/20)*3600,2)
    cost_matrix_moto.loc[row['courier_id'],row['store_id']] = moto_havrs_courier_store
    

#Submatrix of stores vs orders (allow way back)
_cols = ['store_id','order_id','havrs_store_user']
stores_orders = all_posible[_cols].groupby(_cols).count().reset_index()
for i, row in stores_orders.iterrows():
    moto_havrs_store_user = round((row['havrs_store_user']/20)*3600,2)  
    cost_matrix_moto.loc[row['store_id'],row['order_id']] = moto_havrs_store_user
    cost_matrix_moto.loc[row['order_id'],row['store_id']] = moto_havrs_store_user
        
#SubMatrix of stores vs stores: # TODO: optimize with list comprehensions of store ids tuples + lambda function
_cols = ['store_id', 'store_lat', 'store_lng']
stores_dist = all_posible[_cols].groupby(_cols).count().reset_index()
for i, store in stores_dist.iterrows():
    store_lat = store['store_lat']
    store_lng = store['store_lng']
    store_id = store['store_id']
    for j, store_temp in stores_dist.iterrows():
        store_lat_temp = store_temp['store_lat']
        store_lng_temp = store_temp['store_lng']
        store_id_temp = store_temp['store_id']
        if j != i:
            moto_harvs_dist = round((haversine((store_lat, store_lng), (store_lat_temp, store_lng_temp), unit='km')/20)*3600,2)
            #Cost matrix of motorcycles
            cost_matrix_moto.loc[store_id, store_id_temp] = moto_harvs_dist
            cost_matrix_moto.loc[store_id_temp, store_id] = moto_harvs_dist        
        else:
            cost_matrix_moto.loc[store_id, store_id_temp] = 0
            
#SubMatrix of orders vs orders: # TODO: optimize with list comprehensions of orders ids tuples + lambda function
_cols = ['order_id', 'user_lat', 'user_lng']
ords_dist = all_posible[_cols].groupby(_cols).count().reset_index()
for i, order in ords_dist.iterrows():
    order_lat = order['user_lat']
    order_lng = order['user_lng']
    order_id = order['order_id']
    for j, order_temp in ords_dist.iterrows():
        order_lat_temp = order_temp['user_lat']
        order_lng_temp = order_temp['user_lng']
        order_id_temp = order_temp['order_id']
        if j != i:
            moto_harvs_dist = round((haversine((order_lat, order_lng), (order_lat_temp, order_lng_temp), unit='km')/20)*3600,2)
            #Cost matrix of motorcycles
            cost_matrix_moto.loc[order_id, order_id_temp] = moto_harvs_dist
            cost_matrix_moto.loc[order_id_temp, order_id] = moto_harvs_dist
        else:
            cost_matrix_moto.loc[order_id, order_id_temp] = 0

#Start and ending anywhere (Open VRP)
# First row set to 0
depot_row = pd.DataFrame(2401, index=[0], columns=cost_matrix_ids)
cost_matrix_moto = pd.concat([depot_row,cost_matrix_moto])

#First column set to 0
cost_matrix_moto.insert(loc=0,column=0,value=0)

cost_matrix_moto.loc[0,0] = 0

#Distance from depot to rappis is 0, because the route starts in them
#--- Distance from depot to stores or users/orders is the 2401 
#---- since I can't reach them before going to the rappi
for courier_id in cours_ids:
    cost_matrix_moto.loc[0,courier_id] = 0
    
#Distance from the users to the depot is 0 since i can only end when I've arrive a user
for order_id in ords_ids:
    cost_matrix_moto.loc[order_id,0] = 0
    
cost_matrix = cost_matrix_moto.to_numpy()
# Depot to depot is 0
cost_matrix_moto

Wall time: 2.16 s


,0.0,724468.0,219582.0,133284.0,799965.0,392492.0,232622.0,739216.0,820048.0,722928.0,...,66128565.0,66124395.0,66128817.0,66127709.0,66113201.0,66127169.0,66127156.0,66131596.0,66130413.0,66132360.0
0.0,0,0,0,0,0,0,0,0,0,0,...,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00
724468.0,0,0,2401,2401,2401,2401,2401,2401,2401,2401,...,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00
219582.0,0,2401,0,2401,2401,2401,2401,2401,2401,2401,...,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00
133284.0,0,2401,2401,0,2401,2401,2401,2401,2401,2401,...,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00
799965.0,0,2401,2401,2401,0,2401,2401,2401,2401,2401,...,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00,2401.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66127169.0,0,2401,2401,2401,2401,2401,2401,2401,2401,2401,...,1263.74,1653.00,1566.12,755.99,541.70,0.00,493.15,975.93,2560.83,2602.77
66127156.0,0,2401,2401,2401,2401,2401,2401,2401,2401,2401,...,805.89,1209.80,1165.82,1156.48,901.69,493.15,0.00,669.20,2106.94,2155.02
66131596.0,0,2401,2401,2401,2401,2401,2401,2401,2401,2401,...,509.73,774.81,620.94,1729.20,1496.84,975.93,669.20,0.00,2234.75,2301.38
66130413.0,0,2401,2401,2401,2401,2401,2401,2401,2401,2401,...,1746.48,1752.21,1985.77,2947.79,2711.51,2560.83,2106.94,2234.75,0.00,85.08


### 2.2 Construction of the time windows, pickup & deliveries tuples arrays, the service times list AND the demands list

In [18]:
%%time
#Dictionary that has as key -> store, courier or user/order id, 
# and as value its index in the times costs matrix
_ids_dic = {}
for i, col in enumerate(cost_matrix_moto.columns): 
    _ids_dic[col] = i

pickup_deliveries = []
tw_tuples_arr = []
service_times = []
demands = []

# DEPOT
demands.append(0)
service_times.append(0)
tw_tuples_arr.append((0,2400))

# ======= TIME WINDOWS  & SERVICE TIMES LIST  && CAPACITIES =======

# tws for rappis
for i, courier_id in enumerate(cours_ids):
    tw_tuples_arr.append((0,5)) #Limit the time window of the rappi -> can't to come back
    service_times.append(0) #Rappis have no service time
    demands.append(0)
    
# tws for stores
_cols = ['store_id', 'store_eta_lb', 'store_eta_ub','store_service_time']
stores_tws = all_posible[_cols].groupby(_cols).count().reset_index()
for store_id in sts_ids:
    store_eta_lb = stores_tws.loc[(stores_tws['store_id'] == store_id), "store_eta_lb"].iloc[0]
    store_eta_ub = stores_tws.loc[(stores_tws['store_id'] == store_id), "store_eta_ub"].iloc[0]
    store_serv_time = stores_tws.loc[(stores_tws['store_id'] == store_id), "store_service_time"].iloc[0]
    tw_tuples_arr.append((int(store_eta_lb), int(store_eta_ub))) # Append time window tuple
    service_times.append(int(store_serv_time)) # Append service time in correct index
    demands.append(0)
    
# tws for orders/users
_cols = ['order_id', 'user_eta_lb', 'user_eta_ub', 'user_service_time']
users_tws = all_posible[_cols].groupby(_cols).count().reset_index()
for user_id in ords_ids:
    user_eta_lb = users_tws.loc[(users_tws['order_id'] == user_id), "user_eta_lb"].iloc[0]
    user_eta_ub = users_tws.loc[(users_tws['order_id'] == user_id), "user_eta_ub"].iloc[0]
    user_serv_time = users_tws.loc[(users_tws['order_id'] == user_id), "user_service_time"].iloc[0]
    tw_tuples_arr.append((int(user_eta_lb), int(user_eta_ub))) # Append time window tuple
    service_times.append(int(user_serv_time))# Append service time in correct index
    demands.append(1)
    
# ======= PICKUP & DELIVERIES =======

_cols = ['order_id', 'store_id']
orders_stores = all_posible[_cols].groupby(_cols).count().reset_index()
for i, order_store in orders_stores.iterrows():
    store_id = order_store['store_id']
    order_id = order_store['order_id']
    pickup_deliveries.append((_ids_dic[store_id], _ids_dic[order_id] ))
    
# ====== CAPACITIES =======
capacities = [3 for i in range(len(cours_ids))]

Wall time: 185 ms


In [19]:
## Lets see result
print(f"Time costs matrix:\n{cost_matrix}\n")
print(f"Time windows:\n{tw_tuples_arr}\n")
print(f"Pickup & Deliveries:\n{pickup_deliveries}\n")
print(f"Service times list:\n{service_times}\n")
print(f"Demands list:\n{demands}\n")
print(f"Capacities list:\n{capacities}\n")
print(f"Number of vehicles:\n{len(cours_ids)}\n")
print(f"Number of stores :\n{len(sts_ids)}\n")
print(f"Number of orders :\n{len(ords_ids)}\n")

# cost_matrix, tw_tuples_arr, pickup_deliveries, service_times, len(cours_ids), len(ords_ids)

Time costs matrix:
[[   0.      0.      0.   ... 2401.   2401.   2401.  ]
 [   0.      0.   2401.   ... 2401.   2401.   2401.  ]
 [   0.   2401.      0.   ... 2401.   2401.   2401.  ]
 ...
 [   0.   2401.   2401.   ...    0.   2234.75 2301.38]
 [   0.   2401.   2401.   ... 2234.75    0.     85.08]
 [   0.   2401.   2401.   ... 2301.38   85.08    0.  ]]

Time windows:
[(0, 2400), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (0, 5), (122, 1322), (91, 1291), (53, 1253), (0, 975), (0, 979), (0, 781), (0, 691), (0, 641), (0, 746), (0, 600), (0, 600), (0, 600), (0, 600), (0, 600), (94, 1294), (0, 963), (0, 600), (0, 764), (0, 1107), (25, 1225), (123, 1323), (0, 1048), (0, 2262), (0, 1458), (0, 976), (0, 2141), (0, 600), (0, 600), (0, 1287), (0, 1824), (0, 2002), (0, 2299), (0, 1155), (0, 1261), (0, 1099), (0, 600), (0, 1178), (0, 827), (0, 600), (0, 624), (0, 618), (0, 2043), (0, 1682), (0, 227

## 3. Execute the solution

In [20]:
data_model = {}
data_model['time_matrix'] = cost_matrix
data_model['time_windows'] = tw_tuples_arr
data_model['pickups_deliveries'] = pickup_deliveries
data_model['service_times'] = service_times
data_model['num_vehicles'] = len(cours_ids)
data_model['demands'] = demands
data_model['capacities'] = capacities
data_model['depot'] = 0
data_model['real_ids'] = cost_matrix_ids


print("How many time do you want the model to execute (seconds) ?")
print("Recommended: [15, 120] seconds")
data_model['exe_time'] = int(input())

How many time do you want the model to execute (seconds) ?
Recommended: [15, 120] seconds


 15


In [21]:
%%time
"""Vehicles Routing Problem (VRP) with Time Windows."""

from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model(data_model):
    """Stores the data for the problem."""
    data = data_model
    return data

# ================================== SOLUTION PRINTER ===============================================
def print_solution(data, manager, routing, solution, penalty):
    # Display dropped nodes.
    total_time_penalty = 0
    total_drops = 0
    dropped_nodes = 'Dropped nodes:'
    for node in range(routing.Size()):
        #print(f"Node: {manager.IndexToNode(node)} {data['demands'][manager.IndexToNode(node)]}")
        if routing.IsStart(node) or routing.IsEnd(node):
            continue
        if solution.Value(routing.NextVar(node)) == node and data['demands'][manager.IndexToNode(node)] > 0:
            dropped_nodes += ' {0} (id: {1})'.format(manager.IndexToNode(node), data['real_ids'][manager.IndexToNode(node)])
            total_drops+=1
            total_time_penalty+=penalty
        
    """Prints solution on console."""
    total_time=0
    time_dimension = routing.GetDimensionOrDie('Time')
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {0} (id: {1}):\n'.format(vehicle_id+1, data['real_ids'][manager.IndexToNode(vehicle_id)])
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            time_var = time_dimension.CumulVar(index)
            # Tiempo de la ruta
            plan_output += 'Node: {0} time window({1}s,{2}s), '.format(manager.IndexToNode(index), 
                                                          solution.Min(time_var),
                                                          solution.Max(time_var))
            # Carga del courier en la ruta
            route_load += data['demands'][node_index]
            plan_output += 'delivered orders({0}) -> '.format(route_load)
            index = solution.Value(routing.NextVar(index))
            
        time_var = time_dimension.CumulVar(index)
        plan_output += 'Node: {0} time window({1}s,{2}s), '.format(manager.IndexToNode(index),
                                                                   solution.Min(time_var),
                                                                   solution.Max(time_var))
        plan_output += '{0} delivered orders({1})\n'.format(manager.IndexToNode(index),
                                                 route_load)
        plan_output += 'Time of the route: {}secs\n'.format(solution.Min(time_var))
        plan_output += 'Total delivered orders of the route: {}\n'.format(route_load)
        print(plan_output)
        total_time += solution.Min(time_var)
        total_time_penalty += solution.Min(time_var)
        
    print("===========================================================================\nResults")
    print(dropped_nodes + f" - Total: {total_drops}\n")
    print('Total time of all routes: {}secs'.format(total_time))
    print('Total time of all routes (Penalty -> +{0}secs): {1}secs'.format(penalty, total_time_penalty))


def main(data_model):
    """Solve the VRP with time windows."""
    # Instantiate the data problem.
    data = create_data_model(data_model)
    #print(data)
    print("check model creation")
    
    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def time_callback(from_index, to_index):
        """Returns the travel time between the two nodes."""
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        #print(f"time {from_node} {to_node} {data['time_matrix'][from_node][to_node]} {data['service_times'][to_node]}")
        return data['time_matrix'][from_node][to_node] + data['service_times'][to_node]

    transit_callback_index = routing.RegisterTransitCallback(time_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    print("check define costs")
    
    # Add Capacity constraints
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')
    print("check capacity constraint")
    
    # Add Time limit constraint
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        2400,  # allow waiting time
        2400,  # maximum time per vehicle
        False,  # Don't force start cumul to zero.
        time)
    time_dimension = routing.GetDimensionOrDie(time)
    print("check order time limit")
    
    # Define Transportation Requests.
    for request in data['pickups_deliveries']:
        pickup_index = manager.NodeToIndex(request[0])
        delivery_index = manager.NodeToIndex(request[1])
        #print(f"Pickup: {pickup_index} - Delivery: {delivery_index}")
        routing.AddPickupAndDelivery(pickup_index, delivery_index)
        routing.solver().Add( routing.VehicleVar(pickup_index) == routing.VehicleVar(delivery_index))
        routing.solver().Add(time_dimension.CumulVar(pickup_index) <= time_dimension.CumulVar(delivery_index))
    print("check pickup deliveries")
    
    # Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        if location_idx == 0:
            continue
        index = manager.NodeToIndex(location_idx)
        #print(f"{time_window[0]}, {time_window[1]}, {type(time_window[0])}, {type(time_window[1])}")
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
    print("check time windows")
    
    # Add time window constraints for each vehicle start node.
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(data['time_windows'][0][0],
                                                data['time_windows'][0][1])
    print("check time windows 2.0")
    
    # Allow to drop nodes --- penalty restriction.
    penalty = 3600
    for node in range(1, len(data['time_matrix'])):
        routing.AddDisjunction([manager.NodeToIndex(node)], penalty)
    
    print("check penalties")

    # Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = ( routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)
    search_parameters.local_search_metaheuristic = ( routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
        # For details take a look to the documentation: https://developers.google.com/optimization/routing/routing_options#local_search_options
    search_parameters.time_limit.FromSeconds(data['exe_time'])

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    

    # Print solution on console.
    if solution:
        print("\nSolution founded =================================")
        print_solution(data, manager, routing, solution, penalty)
    else:
        print("Problem has unfeasible solution")


#if __name__ == '__main__':
main(data_model)

check model creation
check define costs
check capacity constraint
check order time limit
check pickup deliveries
check time windows
check time windows 2.0
check penalties

Solution founded =================================
Route for vehicle 1 (id: 724468):
Node: 0 time window(0s,0s), delivered orders(0) -> Node: 1 time window(0s,0s), delivered orders(0) -> Node: 27 time window(280s,280s), delivered orders(0) -> Node: 48 time window(1155s,1155s), delivered orders(1) -> Node: 0 time window(1155s,1155s), 0 delivered orders(1)
Time of the route: 1155secs
Total delivered orders of the route: 1

Route for vehicle 2 (id: 219582):
Node: 0 time window(0s,0s), delivered orders(0) -> Node: 2 time window(0s,0s), delivered orders(0) -> Node: 25 time window(596s,596s), delivered orders(0) -> Node: 49 time window(1610s,1610s), delivered orders(1) -> Node: 0 time window(1610s,1610s), 0 delivered orders(1)
Time of the route: 1610secs
Total delivered orders of the route: 1

Route for vehicle 3 (id: 1332